# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [0]:
## import Pytorch ##############################################################
running_on_google_colab = True

if running_on_google_colab:
  from os.path import exists
  from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
  platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
  cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
  accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

  !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
  
  !pip install Pillow==4.1.1

In [0]:
## Upload Flower_data.zip#######################################################
!wget https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json
!wget https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip flower_data.zip

In [0]:
## Imports here ################################################################
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import torchvision.models as models
import time
from datetime import datetime
from torch.utils.data.sampler import SubsetRandomSampler
from collections import OrderedDict
import torchvision.models as models

#densenet121=models.densenet121(pretrained=True)
densenet161 = models.densenet161(pretrained=True)
#resnet152 = models.resnet152(pretrained=True)

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [0]:
data_dir =  'flower_data'
train_dir = '/train'
valid_dir = '/valid'

In [0]:
# Define your transforms for the training and validation sets
mean =  np.array([0.229, 0.224, 0.225])
std = np.array([0.485, 0.456, 0.406])
batch_size=91
num_workers=32

# percentage of training set to use as validation
valid_size = 0.5

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean,
                                                            std)])
valid_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean,
                                                            std)])

# Load the datasets with ImageFolder
train_set = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
valid_set = datasets.ImageFolder(data_dir + '/valid', transform=valid_transforms)

# obtain validation indices that will be used for test     #####################
num_valid = len(valid_set)
indices = list(range(num_valid))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_valid))
valid_idx, test_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches ################
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

# Using the image datasets and the trainforms, define the dataloaders ##########
# Define batchs = 72 --> 6552/72 --> batch_size = 91 images
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers, shuffle=False, sampler=valid_sampler)
test_loader  = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, num_workers=num_workers, shuffle=False, sampler=test_sampler)

In [0]:
## Show images in train_loader
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    ax.set_title(title)
    
    return ax

In [0]:
# Verify data_train
images, labels = next(iter(train_loader))
imshow(images[1],title=labels[1].item())

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [0]:
# Verify data_train
############## 1
import json
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
print(cat_to_name)
  
############## 2    
class_to_idx = train_set.class_to_idx    
print(class_to_idx)   

############## 3 
cat_label_to_name = {}
for cat, label in class_to_idx.items():
  name = cat_to_name.get(cat)
  cat_label_to_name[label] = name
  
print(cat_label_to_name)  

############## 4 
cat_position = {}
for key, value in class_to_idx.items():
  cat_position[value] = key # para adicionar chave e valor no dict

print(cat_position)  

In [0]:
# Verify data_train ############## amostra teste

images, labels = next(iter(train_loader))
label = labels[1].item()
title = f'Position: {labels[1].item()}\n Folder: {cat_position[label]}\n Flower Name: {cat_label_to_name[label].title()}'
imshow(images[1],title=title)

In [0]:
# change this to the trainloader or testloader 
data_iter = iter(train_loader)
images, labels = data_iter.next()

#images = images.numpy() # convert images to numpy for display
fig = plt.figure(figsize=(20, 9))
nrows, ncols = 2, 5

for index in np.arange(nrows*ncols):
  img = images[index]
  
  label = labels[index].item()
  title = f'Position: {labels[index].item()}\n Folder: {cat_position[label]}\n Name Flower: {cat_label_to_name[label].title()}'
  ax = fig.add_subplot(nrows, ncols, index+1, xticks=[], yticks=[])
  imshow(img,title=title,ax=ax)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [0]:
# Part 1: Pretrained Network
# TODO: Build and train your network
model    = densenet161

In [0]:
model

In [0]:
def Net_L(in_size,ou_size,hd_size,dp_prob=0.5):
  dict=OrderedDict()
  dict['input'] = nn.Linear(in_size, hd_size, bias=True)
  dict['relu'] = nn.ReLU()
  dict['dropout'] = nn.Dropout(dp_prob)
  dict['output'] = nn.Linear(hd_size, ou_size, bias=True)
  dict['Log'] = nn.LogSoftmax(dim=1)
  
  return nn.Sequential(dict)

In [0]:
# Part 1: Feedforward Classifier ################### não mexer

# Hyperparameters for our network
in_size = 2208  # features
ou_size = 102  # output
dp_prob = 0.2   # Dropout module with 0.2 drop probability
hd_size = 1000  # features
lr      = 0.07 # learning_rate 
mom     = 0.9
adam        = optim.Adam(model.classifier.parameters(), lr=lr) # optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
opt         = optim.SGD(model.classifier.parameters(), lr=lr)
mom         = optim.SGD(model.classifier.parameters(), lr=lr, momentum=mom)
criterion_n = nn.NLLLoss()
criterion_c = nn.CrossEntropyLoss()

# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Freeze parameters
for param in model.parameters(): 
    param.requires_grad = False

# Update classifier
model.classifier = Net_L(in_size,ou_size,hd_size,dp_prob)

criterion = nn.NLLLoss()    
model.to(device);
optimizer = optim.Adamax(model.classifier.parameters())
#optimizer = optim.SGD(model.classifier.parameters(),lr=0.01)
#optimizer = optim.SGD(model.classifier.parameters(),lr=0.01, momentum=0.9)

In [11]:
# Part 1: Training the network
# Hyperparameters for training network

epochs = 2                # epochs
steps = 0                 # steps
running_loss = 0          # loss
print_every = 72          # print

    
    ###################
    # train           #
    ###################
train_losses, valid_losses, accuracy_losses = [], [], []

for epoch in range(epochs):
    for images, labels in train_loader:
        steps += 1
        if epoch ==0:
          start = datetime.now()       
        
        # Move input and label tensors to the default device
        images, labels = images.to(device), labels.to(device)        
        
        optimizer.zero_grad()
        
        logps = model.forward(images)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    ###################
    # valid           #
    ###################        
        if steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            model.eval()  # turn dropout off to valid_loader
            with torch.no_grad():
                for images, labels in valid_loader:
                    images, labels = images.to(device), labels.to(device)
                    logps = model.forward(images)
                    batch_loss = criterion(logps, labels)
                    
                    valid_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            train_losses.append(running_loss/len(train_loader))
            valid_losses.append(valid_loss/len(valid_loader))         
            accuracy_losses.append(accuracy/len(valid_loader))

            if epoch == 19:
              end_f = datetime.now()
              
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Valid loss: {valid_loss/len(valid_loader):.3f}.. "
                  f"Accuracy: {(accuracy/len(valid_loader)*100):.3f} %")
                  
            running_loss = 0
            model.train()

Epoch 1/2.. Train loss: 3.114.. Valid loss: 1.754.. Accuracy: 65.001 %
Epoch 2/2.. Train loss: 1.315.. Valid loss: 0.943.. Accuracy: 81.724 %


In [0]:
print(start)
print(end_f)

print(end_f-start)

In [0]:
plt.title('Model: densenet161 \n Epochs = 40 \n Optimizer = Adamax  and mom=  \n dropout = 0.2 and Time = 51 min', fontsize=15)

plt.ylabel('Losses')
plt.xlabel('Epochs')
plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='Validation loss')
plt.plot(accuracy_losses, label='Accuracy')
plt.legend(frameon=False)      
plt.grid(True)

In [0]:
  
    ###################
    # test            #
    ###################

    
    

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
# TODO: Save the checkpoint 

print("Our model: \n\n", model, '\n')
#print("The state dict keys: \n\n", model.state_dict().keys())

In [0]:
# Save state the dict
torch.save(model.state_dict(), 'checkpoint.pth')
state_dict = torch.load('checkpoint.pth')
print(state_dict.keys())

In [0]:
checkpoint = {'in_size': 2208,
              'ou_size': 102,
              'hd_size': 1000,
              'dp_prob': 0.5,
              'state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [0]:
model.classifier = Net_L(checkpoint['in_size'],checkpoint['ou_size'],checkpoint['hd_size'],checkpoint['dp_prob'])

In [0]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model.classifier = nn.Sequential(nn.Linear(fc1_in_size, fc1_ou_size, bias=bias_T),
                                     nn.ReLU(),
                                     nn.Dropout(fc1_dp_prob),
                                     nn.Linear(fc2_in_size, fc2_ou_size, bias=bias_T),
                                     nn.LogSoftmax(dim=1))
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

In [0]:
model

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [0]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [0]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [0]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [0]:
# TODO: Display an image along with the top 5 classes